# MMTracking의 DeepSORT를 이용한 Object Tracking

In [1]:
#코랩의 pytorch 버전이 2.0으로 upgrade됨.
import torch
print(torch.__version__)

2.1.0+cu118


In [2]:
#1분 소모

#현재 mmdetection이 pytorch 2.x를 지원하지 않음. pytorch 1.13+ CUDA 116 환경으로 코랩 커널 downgrade
# downgrade 설치시 ERROR: pip's dependency resolver... 부분은 신경쓰지 않아도 됨.
!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 34.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu118
    Uninstalling torchvision-0.16.0+cu118:
      Successfully uninstalled torchvision-0.16.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.0+cu116 which is incompatible.
torchtext 0.16.0 requires to

In [3]:
# mmcv를 위해서 mmcv-full을 먼저 설치해야 함. https://mmcv.readthedocs.io/en/latest/get_started/installation.html 설치 과정 참조.
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 5.3 MB/s eta 0:00:00


In [4]:
# 2023년 4월 6일 기준으로 mmdetection이 3.0으로 upgrade됨. 실습 코드는 mmdetection 2.x 기준임.
# mmdetection 2.x branch의 소스코드 기반으로 mmdetection 설치 필요.
!git clone --branch 2.x https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

Cloning into 'mmdetection'...
remote: Enumerating objects: 37679, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 37679 (delta 114), reused 171 (delta 71), pack-reused 37399
Receiving objects: 100% (37679/37679), 63.12 MiB | 21.67 MiB/s, done.
Resolving deltas: 100% (25937/25937), done.
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-p

In [1]:
# 아래를 수행하기 전에 kernel을 restart 해야 함(런타임 다시시작)
from mmdet.apis import init_detector, inference_detector
import mmcv

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
# 데이터가 저장 된 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# install MMDetection
!pip install mmdet==2.28.2

In [5]:
!pwd

/content


In [6]:
# clone the MMTracking repository
!git clone https://github.com/open-mmlab/mmtracking.git

%cd mmtracking

Cloning into 'mmtracking'...
remote: Enumerating objects: 7927, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 7927 (delta 0), reused 5 (delta 0), pack-reused 7915
Receiving objects: 100% (7927/7927), 2.91 MiB | 7.68 MiB/s, done.
Resolving deltas: 100% (5021/5021), done.
/content/mmtracking


In [7]:
!pwd

/content/mmtracking


In [8]:
# 1분 소모
# 설치시 ERROR: pip's dependency resolver... 부분은 신경쓰지 않아도 됨.
# install MMTracking and its dependencies
!pip install -r requirements/build.txt
!pip install -e .
# used to MOT evaluation
!pip install git+https://github.com/JonathonLuiten/TrackEval.git

Obtaining file:///content/mmtracking
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.8/648.8 kB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 61.1 MB/s eta 0:00:00
  Created wheel for attributee: filename=attributee-0.1.8-py3-none-any.whl size=12859 sha256=219f727aa89e5d45fc2e605e86a0344650c25828195bfea86972e1b2dae3a49d
  Stored in directory: /root/.cache/pip/wheels/0a/e2/ab/3292c3de25a615d3ef2011d65e68f103c025d82149ac6c2f14
  Creat

  Cloning https://github.com/JonathonLuiten/TrackEval.git to /tmp/pip-req-build-wj8uyhjh
  Running command git clone --filter=blob:none --quiet https://github.com/JonathonLuiten/TrackEval.git /tmp/pip-req-build-wj8uyhjh
  Resolved https://github.com/JonathonLuiten/TrackEval.git to commit 12c8791b303e0a0b50f753af204249e622d0281a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trackeval: filename=trackeval-1.0.dev1-py3-none-any.whl size=149965 sha256=dfe9e956c128305b0243b2e81729dd9f6f571abffc6310b38e77a32d5b5b7a77
  Stored in directory: /tmp/pip-ephem-wheel-cache-dvor326j/wheels/bf/6f/b9/d7f59588070500e34ef9a3377da7dbff1ea5ead3cdbca7c0aa
Successfully built trackeval


In [9]:
from mmcv import collect_env
collect_env() # 설정된 환경 확인

{'sys.platform': 'linux',
 'Python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]',
 'CUDA available': True,
 'GPU 0': 'Tesla T4',
 'CUDA_HOME': '/usr/local/cuda',
 'NVCC': 'Cuda compilation tools, release 11.8, V11.8.89',
 'GCC': 'x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0',
 'PyTorch': '1.13.0+cu116',
 'PyTorch compiling details': 'PyTorch built with:\n  - GCC 9.3\n  - C++ Version: 201402\n  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications\n  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)\n  - OpenMP 201511 (a.k.a. OpenMP 4.5)\n  - LAPACK is enabled (usually provided by MKL)\n  - NNPACK is enabled\n  - CPU capability usage: AVX2\n  - CUDA Runtime 11.6\n  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;a

In [10]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.13.0+cu116 True


In [11]:
# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

11.6
GCC 9.3


In [12]:
# Check MMDetection installation
import mmdet
print(mmdet.__version__)

2.28.2


In [13]:
# Check MMTracking installation
import mmtrack
print(mmtrack.__version__)

0.14.0


In [14]:
# 49초 소모
# unset the proxy for downloading the pretrained models (optional)
!unset https_proxy
!unset http_proxy

# download checkpoints
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r50_dc5_1x_imagenetvid/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth -P ./checkpoints
!wget -c https://download.openmmlab.com/mmtracking/sot/siamese_rpn/siamese_rpn_r50_1x_lasot/siamese_rpn_r50_1x_lasot_20211203_151612-da4b3c66.pth -P ./checkpoints
!wget -c https://download.openmmlab.com/mmtracking/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth -P ./checkpoints

'''
Siamese Network 는 (이하 샴 네트워크) 다루어야하는 클래스의 종류가 매우 많고,
특정 클래스에 대한 사진을 대량으로 구할 수 없을 때 머신러닝을 활용하여 그
클래스를 구분해내기 위하여 고안된 네트워크이다.
'''

--2023-11-30 09:50:40--  https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r50_dc5_1x_imagenetvid/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 192.169.122.216, 192.169.122.217, 192.169.122.210, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|192.169.122.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282801031 (270M) [application/octet-stream]
Saving to: ‘./checkpoints/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth’

selsa_faster_rcnn_r 100%[===================>] 269.70M  52.7MB/s    in 5.2s    

2023-11-30 09:50:45 (52.0 MB/s) - ‘./checkpoints/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth’ saved [282801031/282801031]

--2023-11-30 09:50:46--  https://download.openmmlab.com/mmtracking/sot/siamese_rpn/siamese_rpn_r50_1x_lasot/siamese_rpn_r50_1x_lasot_20211203_151612-da4b3c66.pth
Resolving 

'\nSiamese Network 는 (이하 샴 네트워크) 다루어야하는 클래스의 종류가 매우 많고,\n특정 클래스에 대한 사진을 대량으로 구할 수 없을 때 머신러닝을 활용하여 그\n클래스를 구분해내기 위하여 고안된 네트워크이다.\n'

In [15]:
import tempfile

print(tempfile.TemporaryDirectory())

<TemporaryDirectory '/tmp/tmp1wxr6f8c'>


In [16]:
print(tempfile.TemporaryDirectory().name)

/tmp/tmpcr5l4k8f


In [17]:
!pwd

/content/mmtracking


In [18]:
!mkdir data

In [19]:
!mkdir data/MOT17_tiny

In [20]:
%cd /content

/content


In [21]:
!pwd

/content


In [22]:
!unzip -qq '/content/gdrive/MyDrive/Colab Notebooks/REAL_MOT17_tiny.zip' -d '/content/mmtracking/data/MOT17_tiny'

In [ ]:
'''
gt.txt.는 9가지 속성 존재
gt.txt => Frame number, ID number, BBox left, BBox top, BBox width, BBox height, Confidence Score, Class, Visibility(가시성)

det.txt는 7개의 속성 존재(/det/det.txt 파일은 Re-identification을 위한 Annotation은 제외된채로 Detection만을 위한 Annotation이 되어 있다)
det.txt => imageFileNumber(FrameNumber), id(존재하지 않으므로 모두 -1값을 가짐), BBoxTop, BBoxLeft, BBoxWidth, BBoxHeight, Confidence Score
'''

In [23]:
%cd mmtracking

/content/mmtracking


In [24]:
'''
이미지 파일 개수는 244장
'''
import os

folder_path = "/content/mmtracking/data/MOT17_tiny/train/MOT17-02-FRCNN/img1"
file_list = os.listdir(folder_path)
file_count = len(file_list)
print(file_count)

244


In [ ]:
# convert the dataset to coco format

# !rm -rf ./data/MOT17_tiny/annotations
# !python ./tools/convert_datasets/mot/mot2coco.py -i ./data/MOT17_tiny/ -o ./data/MOT17_tiny/annotations --split-train --convert-det

In [ ]:
'''
Person Re-Identification: 복수의 다양한 각도 시점에서 찍은 사진에서 동일 인물을 찾기
'''
# crop pedestrian patches from the original dataset for training reid model. It may take a few minutes.

# !rm -rf ./data/MOT17_tiny/reid
# !python ./tools/convert_datasets/mot/mot2reid.py -i ./data/MOT17_tiny/ -o ./data/MOT17_tiny/reid --val-split 0.9 --vis-threshold 0.8

In [25]:
!mkdir tutorial_exps

In [26]:
!mkdir tutorial_exps/detector

In [ ]:
'''

이후
/content/mmtracking/configs/det/faster-rcnn_r50_fpn_4e_mot17-half.py에서 아래 복붙


USE_MMDET = True
_base_ = [
    '../_base_/models/faster_rcnn_r50_fpn.py',
    '../_base_/datasets/coco_detection.py', '../_base_/default_runtime.py'
]
model = dict(
    detector=dict(
        rpn_head=dict(bbox_coder=dict(clip_border=False)),
        roi_head=dict(
            bbox_head=dict(bbox_coder=dict(clip_border=False), num_classes=1)),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=  # noqa: E251
            '/content/gdrive/MyDrive/Colab Notebooks/aircraft_work_dir/epoch_5.pth'  # noqa: E501
        )))
# learning policy
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=100,
    warmup_ratio=1.0 / 100,
    step=[3])
# runtime settings
total_epochs = 4






이후
/content/mmtracking/configs/_base_/datasets/coco_detection.py'로 들어가서 아래 복붙


dataset_type = 'CocoDataset'
data_root = 'data/MOT17_tiny/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
      type=dataset_type,
      ann_file=data_root + 'annotations/train_cocoformat.json',
      img_prefix=data_root + 'train',
      classes=('pedestrian', ),
      pipeline=train_pipeline),
    val=dict(
      type=dataset_type,
      ann_file=data_root + 'annotations/half-val_cocoformat.json',
      img_prefix=data_root + 'train',
      classes=('pedestrian', ),
      pipeline=test_pipeline),
    test=dict(
      type=dataset_type,
      ann_file=data_root + 'annotations/half-val_cocoformat.json',
      img_prefix=data_root + 'train',
      classes=('pedestrian', ),
      pipeline=test_pipeline))
evaluation = dict(interval=1, metric='bbox')

'''

In [ ]:
'''

/content/mmtracking/data/MOT17_tiny/reid/meta/train_9.txt

위 경로에 아래 경로의 내용 복붙

/content/mmtracking/data/MOT17_tiny/reid/meta/val_90.txt

'''

In [34]:
!nvidia-smi # 할당된 GPU 확인

Thu Nov 30 09:23:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     8W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [30]:
import mmcv
from mmdet.apis import set_random_seed

cfg = mmcv.Config.fromfile('./configs/det/faster-rcnn_r50_fpn_4e_mot17-half.py') # 4e는 최대 4번의 epoch를 수행한다는 의미

cfg.model.detector.init_cfg.checkpoint= "/content/gdrive/MyDrive/Colab Notebooks/aircraft_work_dir/epoch_5.pth"

cfg.work_dir = './tutorial_exps/detector'
cfg.seed = 0
set_random_seed(0, deterministic=False)
# step을 10으로 지정
cfg.log_config.interval = 10
# 1번 마다 한번씩 로그를 작성(학습시 cell에 출력 됨, None.log.json에도 저장됨)
cfg.log_config.max_epochs = 1
cfg.gpu_ids = range(1)
cfg.device='cuda'

print(cfg)

Config (path: ./configs/det/faster-rcnn_r50_fpn_4e_mot17-half.py): {'model': {'detector': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0], 'clip_border': False}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'SmoothL1Loss', 'beta': 0.1111111111111111, 'loss_weight': 1.0}}, 

In [31]:
import os.path as osp

from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmdet.models import build_detector as build_model

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

model = build_model(cfg.model.detector)
# model.init_weights()
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [32]:
datasets

[
 CocoDataset Train dataset with number of images 215, and instance counts: 
 +----------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 | category       | count | category | count | category | count | category | count | category | count |
 +----------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 |                |       |          |       |          |       |          |       |          |       |
 | 0 [pedestrian] | 988   |          |       |          |       |          |       |          |       |
 +----------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+]

In [33]:
print(datasets[0].CLASSES)

('pedestrian',)


In [34]:
cfg.data.train.classes

('pedestrian',)

In [35]:
'''
[###### Detection Model 학습 ######]
'''
# steps: 10
# epoch:4회 => 4분 소모
# samples_per_gpu는 default 2의 값을 가지므로 batch size는 gpu개수(1) * samples_per_gpu(2) == 2가 된다. 즉 약 200장의 학습 데이터 이미지를 2개의 Batch(각각 약 100장)로 나눠서 학습을 수행

train_model(model, datasets, cfg)

/usr/local/lib/python3.10/dist-packages/mmdet-2.28.2-py3.10.egg/mmdet/utils/compat_config.py:28: UserWarning: config is now expected to have a `runner` section, please set `runner` in your config.
  warnings.warn(
2023-11-30 09:56:22,774 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-11-30 09:56:22,778 - mmdet - INFO - Start running, host: root@7af52c98b43c, work_dir: /content/mmtracking/tutorial_exps/detector
2023-11-30 09:56:22,779 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW      

In [36]:
!nvidia-smi # 할당된 GPU 확인

Thu Nov 30 10:05:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |   5613MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
'''

이후
/content/mmtracking/configs/_base_/datasets/mot_challenge_reid.py 경로에서 아래 복붙하기

dataset_type = 'ReIDDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(
        type='SeqResize',
        img_scale=(128, 256),
        share_params=False,
        keep_ratio=False,
        bbox_clip_border=False,
        override=False),
    dict(
        type='SeqRandomFlip',
        share_params=False,
        flip_ratio=0.5,
        direction='horizontal'),
    dict(type='SeqNormalize', **img_norm_cfg),
    dict(type='VideoCollect', keys=['img', 'gt_label']),
    dict(type='ReIDFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', img_scale=(128, 256), keep_ratio=False),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'], meta_keys=[])
]
data_root = 'data/MOT17_tiny/'
data = dict(
    samples_per_gpu=1,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        triplet_sampler=dict(num_ids=8, ins_per_id=4),
        data_prefix=data_root + 'reid/imgs',
        ann_file=data_root + 'reid/meta/train.txt',
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        triplet_sampler=None,
        data_prefix=data_root + 'reid/imgs',
        ann_file=data_root + 'reid/meta/val_90.txt',
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        triplet_sampler=None,
        data_prefix=data_root + 'reid/imgs',
        ann_file=data_root + 'reid/meta/val_90.txt',
        pipeline=test_pipeline))
evaluation = dict(interval=1, metric='mAP')

'''

In [52]:
import mmcv
from mmdet.apis import set_random_seed

cfg = mmcv.Config.fromfile('./configs/reid/resnet50_b32x8_MOT17.py')

# learning policy
cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=200,
    warmup_ratio=1.0 / 200,
    step=[1])
cfg.total_epochs = 2

cfg.data.train.triplet_sampler["num_ids"] = 5  # (이 부분이 중요) => 한 프레임(이미지)당 나타나는 고유한 객체의 수를 의미(각 고유한 객체는 고유한 id를 가진다)
cfg.data.train.triplet_sampler["ins_per_id"] = 1 # (이 부분이 중요) => 한 id당 이미지의 개수


cfg.work_dir = './tutorial_exps/reid'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
print(cfg)

Config (path: ./configs/reid/resnet50_b32x8_MOT17.py): {'dataset_type': 'ReIDDataset', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'train_pipeline': [{'type': 'LoadMultiImagesFromFile', 'to_float32': True}, {'type': 'SeqResize', 'img_scale': (128, 256), 'share_params': False, 'keep_ratio': False, 'bbox_clip_border': False, 'override': False}, {'type': 'SeqRandomFlip', 'share_params': False, 'flip_ratio': 0.5, 'direction': 'horizontal'}, {'type': 'SeqNormalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'VideoCollect', 'keys': ['img', 'gt_label']}, {'type': 'ReIDFormatBundle'}], 'test_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'Resize', 'img_scale': (128, 256), 'keep_ratio': False}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img'], 'meta

In [53]:
from mmtrack.datasets import build_dataset
from mmdet.apis import train_detector as train_model
from mmtrack.models import build_reid as build_model

model = build_model(cfg.model.reid)
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
datasets[0].CLASSES = ("pedestrian", )
model.CLASSES = datasets[0].CLASSES

2023-11-30 10:17:54,342 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth'}
2023-11-30 10:17:54,344 - mmcv - INFO - load model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2023-11-30 10:17:54,345 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth
2023-11-30 10:17:54,425 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

missing keys in source state_dict: head.fcs.0.fc.weight, head.fcs.0.fc.bias, head.fcs.0.bn.weight, head.fcs.0.bn.bias, head.fcs.0.bn.running_mean, head.fcs.0.bn.running_var, head.fc_out.weight, head.fc_out.bias, head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.

In [54]:
datasets

In [43]:
datasets[0][0]

{'img': DataContainer(tensor([[[[-0.2342, -0.2342, -0.2342,  ..., -0.2342, -0.2342, -0.2342],
           [-0.2342, -0.2342, -0.2342,  ..., -0.2342, -0.2342, -0.2342],
           [-0.2319, -0.2319, -0.2319,  ..., -0.2342, -0.2342, -0.2342],
           ...,
           [-0.2319, -0.2204, -0.2168,  ..., -0.1999, -0.1999, -0.1999],
           [-0.2342, -0.2227, -0.2171,  ..., -0.1999, -0.1999, -0.1999],
           [-0.2342, -0.2227, -0.2171,  ..., -0.1999, -0.1999, -0.1999]],
 
          [[ 1.0105,  1.0105,  1.0105,  ...,  1.0105,  1.0105,  1.0105],
           [ 1.0105,  1.0105,  1.0105,  ...,  1.0105,  1.0105,  1.0105],
           [ 1.0129,  1.0129,  1.0129,  ...,  1.0105,  1.0105,  1.0105],
           ...,
           [ 1.0129,  1.0246,  1.0282,  ...,  1.0455,  1.0455,  1.0455],
           [ 1.0105,  1.0223,  1.0280,  ...,  1.0455,  1.0455,  1.0455],
           [ 1.0105,  1.0223,  1.0280,  ...,  1.0455,  1.0455,  1.0455]],
 
          [[ 1.7163,  1.7163,  1.7163,  ...,  1.7163,  1.7163,  1

In [ ]:
print(cfg)

Config (path: ./configs/reid/resnet50_b32x8_MOT17.py): {'dataset_type': 'ReIDDataset', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'train_pipeline': [{'type': 'LoadMultiImagesFromFile', 'to_float32': True}, {'type': 'SeqResize', 'img_scale': (128, 256), 'share_params': False, 'keep_ratio': False, 'bbox_clip_border': False, 'override': False}, {'type': 'SeqRandomFlip', 'share_params': False, 'flip_ratio': 0.5, 'direction': 'horizontal'}, {'type': 'SeqNormalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'VideoCollect', 'keys': ['img', 'gt_label']}, {'type': 'ReIDFormatBundle'}], 'test_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'Resize', 'img_scale': (128, 256), 'keep_ratio': False}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img'], 'meta

In [ ]:
model.state_dict

<bound method Module.state_dict of BaseReID(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (

In [44]:
cfg.data.train

{'type': 'ReIDDataset',
 'triplet_sampler': {'num_ids': 5, 'ins_per_id': 4},
 'data_prefix': 'data/MOT17_tiny/reid/imgs',
 'ann_file': 'data/MOT17_tiny/reid/meta/train.txt',
 'pipeline': [{'type': 'LoadMultiImagesFromFile', 'to_float32': True},
  {'type': 'SeqResize',
   'img_scale': (128, 256),
   'share_params': False,
   'keep_ratio': False,
   'bbox_clip_border': False,
   'override': False},
  {'type': 'SeqRandomFlip',
   'share_params': False,
   'flip_ratio': 0.5,
   'direction': 'horizontal'},
  {'type': 'SeqNormalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'VideoCollect', 'keys': ['img', 'gt_label']},
  {'type': 'ReIDFormatBundle'}]}

In [45]:
print(datasets[0].CLASSES)

('pedestrian',)


In [46]:
print(model.CLASSES)

('pedestrian',)


In [47]:
!nvidia-smi # 할당된 GPU 확인

Thu Nov 30 10:07:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |   5613MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [55]:
'''
###### ReID Model 학습 ######
'''
# epoch:2회 => 5분 소모

train_model(model, datasets, cfg)
# train_model(model, datasets, cfg, validate=False)

2023-11-30 10:18:05,200 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-11-30 10:18:05,204 - mmdet - INFO - Start running, host: root@7af52c98b43c, work_dir: /content/mmtracking/tutorial_exps/reid
2023-11-30 10:18:05,205 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook                     
(LOW         ) IterTimerHook      

In [ ]:
'''

/content/mmtracking/configs/_base_/datasets/mot_challenge.py


위 경로에서 아래 내용 복붙


# dataset settings
dataset_type = 'MOTChallengeDataset'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadMultiImagesFromFile', to_float32=True),
    dict(type='SeqLoadAnnotations', with_bbox=True, with_track=True),
    dict(
        type='SeqResize',
        img_scale=(1088, 1088),
        share_params=True,
        ratio_range=(0.8, 1.2),
        keep_ratio=True,
        bbox_clip_border=False),
    dict(type='SeqPhotoMetricDistortion', share_params=True),
    dict(
        type='SeqRandomCrop',
        share_params=False,
        crop_size=(1088, 1088),
        bbox_clip_border=False),
    dict(type='SeqRandomFlip', share_params=True, flip_ratio=0.5),
    dict(type='SeqNormalize', **img_norm_cfg),
    dict(type='SeqPad', size_divisor=32),
    dict(type='MatchInstances', skip_nomatch=True),
    dict(
        type='VideoCollect',
        keys=[
            'img', 'gt_bboxes', 'gt_labels', 'gt_match_indices',
            'gt_instance_ids'
        ]),
    dict(type='SeqDefaultFormatBundle', ref_prefix='ref')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1088, 1088),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='VideoCollect', keys=['img'])
        ])
]
data_root = 'data/MOT17_tiny/'
data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        visibility_thr=-1,
        ann_file=data_root + 'annotations/train_cocoformat.json',
        img_prefix=data_root + 'train',
        ref_img_sampler=dict(
            num_ref_imgs=1,
            frame_range=10,
            filter_key_img=True,
            method='uniform'),
        classes=('pedestrian', ),
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'annotations/half-val_cocoformat.json',
        img_prefix=data_root + 'train',
        ref_img_sampler=None,
        classes=('pedestrian', ),
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'annotations/half-val_cocoformat.json',
        img_prefix=data_root + 'train',
        ref_img_sampler=None,
        classes=('pedestrian', ),
        pipeline=test_pipeline))

'''

In [56]:
import mmcv
from mmdet.apis import set_random_seed

cfg = mmcv.Config.fromfile('./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py')

cfg.model.detector.init_cfg.checkpoint = './tutorial_exps/detector/epoch_4.pth'
cfg.model.reid.init_cfg.checkpoint = './tutorial_exps/reid/epoch_2.pth'

cfg.work_dir = './tutorial_exps'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.data.test.test_mode = True
cfg.device='cuda'

print(cfg)

Config (path: ./configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py): {'model': {'detector': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0], 'clip_border': False}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'SmoothL1Loss', 'beta': 0.1111111111111111, 

In [57]:
cfg.data.test

{'type': 'MOTChallengeDataset',
 'ann_file': 'data/MOT17_tiny/annotations/half-val_cocoformat.json',
 'img_prefix': 'data/MOT17_tiny/train',
 'ref_img_sampler': None,
 'classes': ('pedestrian',),
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'MultiScaleFlipAug',
   'img_scale': (1088, 1088),
   'flip': False,
   'transforms': [{'type': 'Resize', 'keep_ratio': True},
    {'type': 'RandomFlip'},
    {'type': 'Normalize',
     'mean': [123.675, 116.28, 103.53],
     'std': [58.395, 57.12, 57.375],
     'to_rgb': True},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'ImageToTensor', 'keys': ['img']},
    {'type': 'VideoCollect', 'keys': ['img']}]}],
 'test_mode': True}

In [58]:
from mmtrack.datasets import build_dataloader
from mmtrack.apis import init_model
from mmcv.parallel import MMDataParallel
from mmtrack.apis import single_gpu_test
from mmtrack.datasets import build_dataset

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [59]:
dataset


MOTChallengeDataset Test dataset with number of images 121, and instance counts: 
+----------------+-------+---------------+-------+----------+-------+----------+-------+----------+-------+
| category       | count | category      | count | category | count | category | count | category | count |
+----------------+-------+---------------+-------+----------+-------+----------+-------+----------+-------+
|                |       |               |       |          |       |          |       |          |       |
| 0 [pedestrian] | 373   | -1 background | 29    |          |       |          |       |          |       |
+----------------+-------+---------------+-------+----------+-------+----------+-------+----------+-------+

In [60]:
# build the model and load checkpoint
model = init_model(cfg)

model = MMDataParallel(model, device_ids=cfg.gpu_ids)
outputs = single_gpu_test(model, data_loader)

2023-11-30 10:21:34,924 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/detector/epoch_4.pth'}
2023-11-30 10:21:34,925 - mmcv - INFO - load model from: ./tutorial_exps/detector/epoch_4.pth
2023-11-30 10:21:34,927 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/detector/epoch_4.pth
2023-11-30 10:21:35,211 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': './tutorial_exps/reid/epoch_2.pth'}
2023-11-30 10:21:35,212 - mmcv - INFO - load model from: ./tutorial_exps/reid/epoch_2.pth
2023-11-30 10:21:35,213 - mmcv - INFO - load checkpoint from local path: ./tutorial_exps/reid/epoch_2.pth
2023-11-30 10:21:35,405 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
The value is the same before and after calling `init_weights` of DeepSORT  
 
2023-11-30 10:21:35,407 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
The value is the same before 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 121/121, 8.8 task/s, elapsed: 14s, ETA:     0s

In [61]:
eval_kwargs = cfg.get('evaluation', {}).copy()
# hard-code way to remove EvalHook args
eval_hook_args = [
    'interval', 'tmpdir', 'start', 'gpu_collect', 'save_best',
    'rule', 'by_epoch'
]
for key in eval_hook_args:
    eval_kwargs.pop(key, None)
eval_kwargs.update(dict(metric=['track']))
metric = dataset.evaluate(outputs, **eval_kwargs)

print(metric)

Evaluate CLEAR MOT results.

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /usr/local/lib/python3.10/dist-packages/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
GT_FOLDER            : /tmp/tmpjgummi6u              
TRACKERS_FOLDER      : /tmp/tmpbb2pa0sn              
OUTPUT_FOLDER        : None                

<Figure size 640x480 with 0 Axes>

In [62]:
# Video Inference 수행
# 3분 소모

from mmtrack.apis import inference_mot, init_model
from mmdet.apis import init_detector, inference_detector

mot_config = './configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py'
input_video = '/content/gdrive/MyDrive/Colab Notebooks/DeepSORT_inference_video/pexels-kindel-media-8824697 (Original).mp4'
imgs = mmcv.VideoReader(input_video)

# build the model from a config file
mot_model = init_model(mot_config, device='cuda:0')
prog_bar = mmcv.ProgressBar(len(imgs))
out_dir = tempfile.TemporaryDirectory()
out_path = out_dir.name

# test and show/save the images(inference)
for i, img in enumerate(imgs):
    result = inference_mot(mot_model, img, frame_id=i)
    mot_model.show_result(
            img,
            result,
            show=False,
            wait_time=int(1000. / imgs.fps),
            out_file=f'{out_path}/{i:06d}.jpg')
    prog_bar.update()

output = '/content/mmtracking/tutorial_exps/Tracking_result_video.mp4'
print(f'\n making the output video at {output} with a FPS of {imgs.fps}')
mmcv.frames2video(out_path, output, fps=imgs.fps, fourcc='mp4v')
out_dir.cleanup()

2023-11-30 10:23:05,727 - mmcv - INFO - initialize FasterRCNN with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth'}
2023-11-30 10:23:05,729 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth
2023-11-30 10:23:05,731 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth
Downloading: "https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth" to /root/.cache/torch/hub/checkpoints/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth


  0%|          | 0.00/158M [00:00<?, ?B/s]

2023-11-30 10:23:11,715 - mmcv - INFO - initialize BaseReID with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth'}
2023-11-30 10:23:11,716 - mmcv - INFO - load model from: https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth
2023-11-30 10:23:11,718 - mmcv - INFO - load checkpoint from http path: https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth
Downloading: "https://download.openmmlab.com/mmtracking/mot/reid/tracktor_reid_r50_iter25245-a452f51f.pth" to /root/.cache/torch/hub/checkpoints/tracktor_reid_r50_iter25245-a452f51f.pth


  0%|          | 0.00/98.4M [00:00<?, ?B/s]

2023-11-30 10:23:19,851 - mmcv - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: head.bn.weight, head.bn.bias, head.bn.running_mean, head.bn.running_var, head.classifier.weight, head.classifier.bias

2023-11-30 10:23:19,875 - mmcv - INFO - 
detector.backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth 
 
2023-11-30 10:23:19,876 - mmcv - INFO - 
detector.backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth 
 
2023-11-30 10:23:19,877 - mmcv - INFO - 
detector.backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmtracking/mot/faster_rcnn/faster-rcnn_r50_fpn_4e_mot17-half-64ee2ed4.pth 
 
2023-11-30 10:23:19,878 - mmcv - INFO - 
detector.backbone.layer1

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 304/304, 3.8 task/s, elapsed: 81s, ETA:     0s
 making the output video at /content/mmtracking/tutorial_exps/Tracking_result_video.mp4 with a FPS of 30.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 304/304, 6.5 task/s, elapsed: 47s, ETA:     0s


In [ ]:
###### 아래 Cell 들은 무시해도 된다 ######

In [ ]:
# gt.txt 파일 수정 (Class를 1에서 0으로 수정)

edited_lines = []

with open("/content/mmtracking/data/MOT17_tiny/train/MOT17-02-FRCNN/gt/gt.txt") as f:
  lines = f.readlines()
  for line in lines:
    tmp = list(line)
    tmp[-6:-5] = str(0)
    line = "".join(tmp)

    edited_lines.append(line)

with open("/content/mmtracking/data/MOT17_tiny/train/MOT17-02-FRCNN/gt/gt.txt", 'w') as f:
  f.writelines(edited_lines)

In [ ]:
'''
/content/mmtracking/data/MOT17_tiny/reid/imgs

위 경로에서

/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000001
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000002
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000003
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000004
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000005

위 폴더명을

/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000000
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000001
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000002
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000003
/content/mmtracking/data/MOT17_tiny/reid/imgs/MOT17-02-FRCNN_000004

로 바꾸기

'''

In [ ]:
edited_lines = []

with open("/content/mmtracking/data/MOT17_tiny/reid/meta/train.txt") as f:
    lines = f.readlines()
    for line in lines:
        # 조건에 따라 원하는 대로 line을 수정
        if (line[15:21] == "000001"):
          line = line.replace("000001", "000000")
          edited_lines.append(line)
        elif (line[15:21] == "000002"):
          line = line.replace("000002", "000001")
          edited_lines.append(line)
        elif (line[15:21] == "000003"):
          line = line.replace("000003", "000002")
          edited_lines.append(line)
        elif (line[15:21] == "000004"):
          line = line.replace("000004", "000003")
          edited_lines.append(line)
        elif (line[15:21] == "000005"):
          line = line.replace("000005", "000004")
          edited_lines.append(line)

with open("/content/mmtracking/data/MOT17_tiny/reid/meta/train.txt", 'w') as f:
    f.writelines(edited_lines)

edited_lines = []

with open("/content/mmtracking/data/MOT17_tiny/reid/meta/train_9.txt") as f:
    lines = f.readlines()
    for line in lines:
        # 조건에 따라 원하는 대로 line을 수정
        if (line[15:21] == "000001"):
          line = line.replace("000001", "000000")
          edited_lines.append(line)
        elif (line[15:21] == "000002"):
          line = line.replace("000002", "000001")
          edited_lines.append(line)
        elif (line[15:21] == "000003"):
          line = line.replace("000003", "000002")
          edited_lines.append(line)
        elif (line[15:21] == "000004"):
          line = line.replace("000004", "000003")
          edited_lines.append(line)
        elif (line[15:21] == "000005"):
          line = line.replace("000005", "000004")
          edited_lines.append(line)

with open("/content/mmtracking/data/MOT17_tiny/reid/meta/train_9.txt", 'w') as f:
    f.writelines(edited_lines)

edited_lines = []

with open("/content/mmtracking/data/MOT17_tiny/reid/meta/val_90.txt") as f:
    lines = f.readlines()
    for line in lines:
        # 조건에 따라 원하는 대로 line을 수정
        if (line[15:21] == "000001"):
          line = line.replace("000001", "000000")
          edited_lines.append(line)
        elif (line[15:21] == "000002"):
          line = line.replace("000002", "000001")
          edited_lines.append(line)
        elif (line[15:21] == "000003"):
          line = line.replace("000003", "000002")
          edited_lines.append(line)
        elif (line[15:21] == "000004"):
          line = line.replace("000004", "000003")
          edited_lines.append(line)
        elif (line[15:21] == "000005"):
          line = line.replace("000005", "000004")
          edited_lines.append(line)

with open("/content/mmtracking/data/MOT17_tiny/reid/meta/val_90.txt", 'w') as f:
    f.writelines(edited_lines)